In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# ✅ Fetch database credentials from environment variables
db_config = {
    "host": os.getenv("DB_HOST"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "database": "mmt"
}

# ✅ Encode the password (handles special characters like '@', ':', etc.)
encoded_password = quote_plus(db_config["password"])

# ✅ Create SQLAlchemy engine
engine = create_engine(f"mysql+pymysql://{db_config['user']}:{encoded_password}@{db_config['host']}/{db_config['database']}")

try:
    # ✅ Fetch car table data
    query_car = "SELECT * FROM car"
    car_df = pd.read_sql(query_car, engine)
    print("\nCar Table:")
    car_df.info()

    # ✅ Fetch car_rental table data
    query_car_rental = "SELECT * FROM rentals"
    car_rental_df = pd.read_sql(query_car_rental, engine)
    print("\nCar Rental Table:")
    car_rental_df.info()

except Exception as e:
    print(f"Error: {e}")



Car Table:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10800 entries, 0 to 10799
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CarID                10800 non-null  int64  
 1   Make                 10800 non-null  object 
 2   Model                10800 non-null  object 
 3   CarType              10800 non-null  object 
 4   Mileage_kmpl         10800 non-null  int64  
 5   Year_Of_Manufacture  10800 non-null  int64  
 6   Price_Per_Day        10800 non-null  float64
 7   City                 10800 non-null  object 
 8   Car_Agency           10800 non-null  object 
 9   Agency_Price         10800 non-null  float64
 10  LocationID           10800 non-null  int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 928.3+ KB

Car Rental Table:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39979 entries, 0 to 39978
Data columns (total 8 columns):
 #   Column           Non-Null Count  

In [3]:
car_df.head()

,CarID,Make,Model,CarType,Mileage_kmpl,Year_Of_Manufacture,Price_Per_Day,City,Car_Agency,Agency_Price,LocationID
0,1,Volkswagen,Volkswagen Tiguan,SUV,15,2007,7500.0,Coimbatore,Budget Wheels Coimbatore,900.0,1
1,2,Volkswagen,Volkswagen Taigun,SUV,19,2021,6500.0,Coimbatore,Budget Wheels Coimbatore,900.0,1
2,3,Volkswagen,Volkswagen Passat,Sedan,18,2010,5800.0,Coimbatore,Budget Wheels Coimbatore,900.0,1
3,4,Volkswagen,Volkswagen Virtus,Sedan,20,2018,5500.0,Coimbatore,Budget Wheels Coimbatore,900.0,1
4,5,Volkswagen,Volkswagen Arteon,Luxury,14,2017,9000.0,Coimbatore,Budget Wheels Coimbatore,900.0,1


In [4]:
car_rental_df.head()

,RentalID,UserID,Pickup_Location,RentalDate,Duration,ReturnDate,CarID,TotalAmount
0,1,0,Visakhapatnam,2019-09-26,3,2019-09-29,9818,28350.0
1,2,0,Shillong,2019-10-10,3,2019-10-13,6755,25200.0
2,3,0,Ranchi,2019-11-14,6,2019-11-20,331,56400.0
3,4,0,Raigarh,2019-12-12,3,2019-12-15,4288,18600.0
4,5,0,Kolkata,2019-12-26,5,2019-12-31,6287,29500.0


In [5]:
# Convert RentalDate and ReturnDate to datetime format
car_rental_df["RentalDate"] = pd.to_datetime(car_rental_df["RentalDate"])
car_rental_df["ReturnDate"] = pd.to_datetime(car_rental_df["ReturnDate"])

# Check for missing values in both datasets
print("Missing values in Car Table:\n", car_df.isnull().sum())
print("\nMissing values in Car Rental Table:\n", car_rental_df.isnull().sum())

# Verify the changes
car_rental_df.head()

Missing values in Car Table:
 CarID                  0
Make                   0
Model                  0
CarType                0
Mileage_kmpl           0
Year_Of_Manufacture    0
Price_Per_Day          0
City                   0
Car_Agency             0
Agency_Price           0
LocationID             0
dtype: int64

Missing values in Car Rental Table:
 RentalID           0
UserID             0
Pickup_Location    0
RentalDate         0
Duration           0
ReturnDate         0
CarID              0
TotalAmount        0
dtype: int64


,RentalID,UserID,Pickup_Location,RentalDate,Duration,ReturnDate,CarID,TotalAmount
0,1,0,Visakhapatnam,2019-09-26,3,2019-09-29,9818,28350.0
1,2,0,Shillong,2019-10-10,3,2019-10-13,6755,25200.0
2,3,0,Ranchi,2019-11-14,6,2019-11-20,331,56400.0
3,4,0,Raigarh,2019-12-12,3,2019-12-15,4288,18600.0
4,5,0,Kolkata,2019-12-26,5,2019-12-31,6287,29500.0


In [6]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# Merge rental data with car data to get city information
rental_merged = car_rental_df.merge(car_df[['CarID', 'City']], on="CarID", how="left")

# Select a user for recommendations
user_index = 0  # Change this to test for other users
user_city = rental_merged.loc[rental_merged.index == user_index, "City"].values[0]

# Filter data for only that city
city_rental_data = rental_merged[rental_merged["City"] == user_city]

# Create a unique index for (UserID, Pickup_Location)
city_rental_data["User_Loc"] = city_rental_data["UserID"].astype(str) + "_" + city_rental_data["Pickup_Location"].astype(str)

# Convert categorical values to numerical indices
user_loc_codes = city_rental_data["User_Loc"].astype("category").cat.codes
car_codes = city_rental_data["CarID"].astype("category").cat.codes

# Values (number of rentals per (User, Pickup_Location, Car))
data = np.ones(len(city_rental_data))

# Create sparse matrix for the city
city_sparse_matrix = csr_matrix((data, (user_loc_codes, car_codes)), 
                               shape=(len(city_rental_data["User_Loc"].unique()), 
                                      len(city_rental_data["CarID"].unique())))

# Print the shape of the sparse matrix
print(f"Sparse Matrix Shape for {user_city}: {city_sparse_matrix.shape}")


Sparse Matrix Shape for Visakhapatnam: (530, 193)


C:\Users\hetvi\AppData\Local\Temp\ipykernel_16208\2235191278.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_rental_data["User_Loc"] = city_rental_data["UserID"].astype(str) + "_" + city_rental_data["Pickup_Location"].astype(str)


In [7]:
# Convert sparse matrix to Compressed Sparse Row format
sparse_matrix_csr = city_sparse_matrix.tocsr()

# Initialize Nearest Neighbors model
knn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)

# Fit the model on city-specific sparse matrix
knn_model.fit(sparse_matrix_csr)


NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

In [8]:
# Function to find similar users
def get_similar_users(user_index, k=10):
    distances, indices = knn_model.kneighbors(sparse_matrix_csr[user_index], n_neighbors=k+1)
    return indices[0][1:], distances[0][1:]  # Exclude self

# Test: Find similar users for the first user in the city
similar_users, similarity_scores = get_similar_users(0, k=10)

# Display similar users and their similarity scores
print("Top 10 similar users in", user_city, ":", similar_users)
print("Similarity scores:", similarity_scores)


Top 10 similar users in Visakhapatnam : [338 227 325 192 313 373 505 506 507 508]
Similarity scores: [0.29289322 0.29289322 0.5        0.5        0.5        0.59175171
 1.         1.         1.         1.        ]


In [9]:
def recommend_cars(user_index, n_recommendations=5):
    # Get similar users
    similar_users, _ = get_similar_users(user_index, k=10)

    # Get aggregated car scores from similar users
    car_scores = np.zeros(city_sparse_matrix.shape[1])
    for similar_user in similar_users:
        car_scores += city_sparse_matrix[similar_user].toarray().flatten()

    # Get top N recommended CarIDs
    recommended_car_indices = np.argsort(car_scores)[::-1][:n_recommendations]
    recommended_cars = city_rental_data["CarID"].astype("category").cat.categories[recommended_car_indices].tolist()

    return recommended_cars

# Get recommended cars for the user in the same city
recommended_cars = recommend_cars(0, n_recommendations=5)

# Display recommended car details
recommended_car_details = car_df[car_df["CarID"].isin(recommended_cars) & (car_df["City"] == user_city)]

print(f"Recommended Cars in {user_city}:")
print(recommended_car_details)


Recommended Cars in Visakhapatnam:
      CarID     Make          Model CarType  Mileage_kmpl  \
9817   9818     Ford    Ford Bronco     SUV            10   
9834   9835  Hyundai  Hyundai Verna   Sedan            19   
9900   9901     Ford   Ford Mustang  Luxury            10   
9970   9971   Toyota   Toyota Camry   Sedan            19   
9989   9990    Honda      Honda NSX  Luxury            10   

      Year_Of_Manufacture  Price_Per_Day           City           Car_Agency  \
9817                 2024         8500.0  Visakhapatnam    Smart Rides Vizag   
9834                 2018         5500.0  Visakhapatnam    Smart Rides Vizag   
9900                 1964        11000.0  Visakhapatnam    Midway Auto Vizag   
9970                 2017         6500.0  Visakhapatnam  Luxury Motion Vizag   
9989                 1999        12500.0  Visakhapatnam  Luxury Motion Vizag   

      Agency_Price  LocationID  
9817         950.0          50  
9834         950.0          50  
9900        1900.0

In [13]:
from sklearn.metrics import ndcg_score
import numpy as np

# Create a dictionary mapping (UserID, Pickup_Location) -> {list of rented CarIDs}
actual_rentals = car_rental_df.groupby(["UserID", "Pickup_Location"])["CarID"].apply(set).to_dict()

# Function to compute Precision@K
def precision_at_k(actual, recommended, k=5):
    if not actual: return 0
    return len(set(actual) & set(recommended[:k])) / k

# Function to compute Recall@K
def recall_at_k(actual, recommended, k=5):
    if not actual: return 0
    return len(set(actual) & set(recommended[:k])) / len(actual)

# Function to compute NDCG@K
def ndcg_at_k(actual, recommended, k=5):
    if not actual or not recommended:
        return 0  # Avoid errors when no actual or recommended items

    actual = list(actual)  # Convert set to list
    recommended = list(recommended[:k])  # Convert to list and limit to top-k

    # Ensure actual has at least 2 items (duplicate if necessary)
    if len(actual) == 1:
        actual.append(actual[0])

    # Ensure recommended has at least 2 items (pad with dummy value if necessary)
    if len(recommended) == 1:
        recommended.append(0)

    # Compute relevance scores
    relevance = [1 if car in actual else 0 for car in recommended]

    return ndcg_score([relevance], [relevance])
# Lists to store scores
precision_scores = []
recall_scores = []
ndcg_scores = []

# Evaluate the model for multiple users
for user_index in range(sparse_matrix_csr.shape[0]):
    # Get user details
    user_id = rental_merged.iloc[user_index]["UserID"]
    user_city = rental_merged.iloc[user_index]["City"]
    pickup_loc = rental_merged.iloc[user_index]["Pickup_Location"]

    # Get similar users
    similar_users, _ = get_similar_users(user_index, k=10)

    # Get recommended cars **only from the same city**
    recommended_cars = []
    for sim_user in similar_users:
        sim_user_cars = car_rental_df[(car_rental_df["UserID"] == sim_user) & (car_rental_df["Pickup_Location"] == pickup_loc)]["CarID"].unique()
        recommended_cars.extend(sim_user_cars)

    # Remove duplicates and limit to top K
    recommended_cars = list(set(recommended_cars))[:5]

    # Retrieve actual rented cars
    actual_cars = actual_rentals.get((user_id, pickup_loc), [])

    # Compute metrics only if there are recommendations
    if recommended_cars:
        precision_scores.append(precision_at_k(actual_cars, recommended_cars))
        recall_scores.append(recall_at_k(actual_cars, recommended_cars))
        ndcg_scores.append(ndcg_at_k(actual_cars, recommended_cars))

# Compute average scores
avg_precision = np.mean(precision_scores) if precision_scores else 0
avg_recall = np.mean(recall_scores) if recall_scores else 0
avg_ndcg = np.mean(ndcg_scores) if ndcg_scores else 0

# Print evaluation results
print(f"Precision@5: {avg_precision:.4f}")
print(f"Recall@5: {avg_recall:.4f}")
print(f"NDCG@5: {avg_ndcg:.4f}")


Precision@5: 0.0083
Recall@5: 0.0253
NDCG@5: 0.0398


improving the above code

In [16]:
rental_merged.columns

Index(['RentalID', 'UserID', 'Pickup_Location', 'RentalDate', 'Duration',
       'ReturnDate', 'CarID', 'TotalAmount', 'City'],
      dtype='object')

In [15]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load the dataset (assuming it's already loaded as `car_rental_df`)
df = rental_merged.copy()  # Copy to avoid modifying original data

# Check for missing values
missing_values = df.isnull().sum()
print("Missing Values:\n", missing_values)

# Fill missing values (if any)
df.fillna(method='ffill', inplace=True)

# Normalize numerical features
scaler = MinMaxScaler()
df[['Price_Per_Day', 'Mileage_kmpl', 'Duration']] = scaler.fit_transform(df[['Price_Per_Day', 'Mileage_kmpl', 'Duration']])

# Display summary
print("\nAfter Normalization:")
print(df[['Price_Per_Day', 'Mileage_kmpl', 'Duration']].describe())


Missing Values:
 RentalID           0
UserID             0
Pickup_Location    0
RentalDate         0
Duration           0
ReturnDate         0
CarID              0
TotalAmount        0
City               0
dtype: int64


C:\Users\hetvi\AppData\Local\Temp\ipykernel_16208\2696789168.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


KeyError: "['Price_Per_Day', 'Mileage_kmpl'] not in index"